# Отработка ошибок свзязанных с запуском кода

- Если после установки необходимых библиотек возникает ошибка то скорее всего не установлен вебдрайвер Chrome для вашей ОС. Как установить вебдрайвер можно ознакомиться по этой ссылке. [selenium-python](https://selenium-python.com/install-chromedriver-chrome?ysclid=ly2ufnhjip111135754)
- Если после установки вебдрайвера код не отрабатывает либо возникает ошибка то требуется проверить не открыт ли в памяти браузер Chrome. Если да то требуется его выгрузить (закрыть). Это связано с тем что данный код отрабатывает в фоновом режиме но при этом использует вебдрайвер Chrome который божет конфликтовать с самим запущенным браузером Chrome.
- Если ошибка попрежнему возникает то требуется проверить или переустановить используемые библиотеки.

In [1]:
#!pip3 install --upgrade pip
#!pip3 install selenium
#!pip3 install beautifulsoup4
#!pip3 install pandas
#!pip3 install regex

In [2]:
import re
import time
import pandas as pd

from datetime import datetime
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
class GoldenPagesParser:

    def __init__(self, task_array):
        # инициализируем массив запросов
        self.task_array = task_array
        # инициализируем вебдрайвер
        options = Options()
        # гиперпараметр для запуска вебдрайвера селениум без графического интерфейса
        options.add_argument('--headless')
        self.driver = Chrome(options=options)
         # параметр максимального времени загрузки страницы сек
        self.upload_wait_time = 10
        # параметр ожидания после прокликивания для загрузки данных для парсинга сек
        self.sleep_time = 1
        self.df = pd.DataFrame({
            'id':pd.Series(dtype='int'),
            'company_name':[],
            'company_second_name':[],
            'company_third_name':[],
            'address':[],
            'landmarks':[],
            'phone_array':[],
            'site_url':[],
            'parsing_url':[],
            'parsing_task':[],
        })

    # функция проверки элементов на присутствие в блоке, в случае отсутствия добавляет пустое значение в случае присутствия очищает текст от лишних пробелов и табуляции
    @staticmethod
    def cleaner(text_block):
        text = re.sub(r'\s+', ' ', text_block.text.replace('\n', '').strip()).replace('\t ', '\t').replace(' \t', '\t') if text_block else None
        return text

    # функция парсинга ячейки
    def cell_parser(self, div, parsing_url, parsing_task):

        line = {}
            
        # забираем данные номера ячейки убираем пробелы табуляцию и лишнюю пунктуацию оставляя только цифры и преобразуем в числовой тип данных
        line['id'] = int(re.sub('[^0-9]', '', div.find('span', attrs={'class':'h3 mb-0 fz-18 clr-blue'}).text)) if div.find('span', attrs={'class':'h3 mb-0 fz-18 clr-blue'}) else None

        line['company_name'] = self.cleaner(div.find('div', attrs={'class':'h3 mb-0'}))
        line['company_second_name'] = self.cleaner(div.find('p', attrs={'class':'d-block mb-0 mx-3 clr-blue'}))
        line['company_third_name'] = self.cleaner(div.find('div', attrs={'class':'gp_name_legal'}))
        line['address'] = self.cleaner(div.find('div', attrs={'class':'gp_wrap_address'}).find('p'))
        line['landmarks'] = self.cleaner(div.find('div', attrs={'class':'gp_wrap_address'}).find('p', attrs={'class':'clr-grey'}))

        # забираем данные блока номеров из ячейки в массив
        phones = div.find_all('a', attrs={'class':'d-flex flex-column'}) if div.find('a', attrs={'class':'d-flex flex-column'}) else []
        # убираем пробелы табуляцию и лишнюю пунктуацию из массива оставляя только цифры и преобразуем в числовой тип данных
        line['phone_array'] = [int(re.sub('[^0-9]', '', phone.get_text(strip=True))) for phone in phones]

        site = div.find('ul', attrs={'class':'gp_listCardIconBottom list-unstyled mb-0 d-flex'}).find_all('li', attrs={'class':'gp_icoActive d-flex d-flex align-items-center justify-content-center'})[-1].get('title') \
            if div.find('ul', attrs={'class':'gp_listCardIconBottom list-unstyled mb-0 d-flex'}).find_all('li', attrs={'class':'gp_icoActive d-flex d-flex align-items-center justify-content-center'}) else []
        # забираем только адрес сайта если он присутствует и массив не пуст чистим адреса сайта для сохранения убираем протокол, хост и путь оставляя только адрес айта
        line['site_url'] = site.replace('https://', '').replace('http://', '').replace('www.', '').split('\\')[0] if site and site not in ['Instagram', 'Telegram', 'Facebook', None] else None

        line['parsing_url'] = parsing_url
        line['parsing_task'] = parsing_task

        return line

    # функция добавления данных со всей страницы в итоговый фрейм
    def page_parser(self, divs, url, parsing_task):
        for div in divs:
            self.df.loc[len(self.df)] = self.cell_parser(div, url, parsing_task)

    #  функция для пркликивания элементов страницы и возврата блока парсинга страницы
    def clicker(self, parsing_url):
        self.driver.get(parsing_url)
        # ожидание загрузки данных
        wait = WebDriverWait(self.driver, self.upload_wait_time)

        # задаем параметры искомого элемента на странице и осуществляем поиск всех элементов соответствующих параметру для дальнейшего взаимодействия если элементов нет то выводим сообщение об отсутствии
        buttons = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//button[contains(text(), 'Показать номер')]")))
        if buttons:
            for button in buttons:
                try:
                    self.driver.execute_script('arguments[0].click();', button)
                except Exception as e:
                    print('Ошибка при нажатии на элемент:', e)
            # время ожидания для загрузки элемента
            time.sleep(self.sleep_time)
        else:
            print('Элементы не найдены на странице.')

        # передаем содержимое вебдрайвера после взаимодействия со страницей из селениума в бьютифул суп для парсинга страницы
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        # ищем все ячейки страницы с требуемыми данными если их нет возвращаем пустой массив, отрабатываются ошибки 404 или заход на страницу за диапазон данных где 404 не появляется
        divs = soup.find('div', attrs={'class':'d-xxl-flex gap-20'}).find_all('section', attrs={'class':'gp_company gp_company_no'}) if soup.find('div', attrs={'class':'d-xxl-flex gap-20'}) else []

        return divs

    # функция парсинга данных всех страниц запроса
    def request_parser(self, url_fix_request_part, parsing_task):
        # номер обрабатываемой страницы
        page_number = 1
        # сборка адреса обрабатывемой страницы
        parsing_url = url_fix_request_part + str(page_number)
        # блок парсинга страницы представлен в виде масссива искомых элементов
        divs = self.clicker(parsing_url)
        while len(divs) > 0:
            self.page_parser(divs, parsing_url, parsing_task)
            print(f'Page N:{page_number}, Task: {parsing_task}')
            # защита от бесконечного цикла на случай если что-то пойдет не так, цикл не может превышать 100 итераций на один запрос поиска
            if page_number == 100:
                break
            page_number += 1
            parsing_url = url_fix_request_part + str(page_number)
            divs = self.clicker(parsing_url)

    # функция генерации адресов страниц
    @staticmethod
    def url_generate(parsing_task):
        url_chenged_request_part = parsing_task.replace(' ', '+')
        url_fix_request_part = 'https://www.goldenpages.uz/search/?region=&s=' + str(url_chenged_request_part) + '&tab=tab1&Page1='
        return (url_fix_request_part, parsing_task)

    # функция парсинга данных всего фрейма запросов
    def task_frame_parser(self):
        driver = self.driver
        for task in self.task_array:
            parsing_task = re.sub(r'\s+', ' ', task.replace('\n', '').strip()).replace('\t ', '\t').replace(' \t', '\t') if task else ''
            # через * вызываем кортеж передаваемых аргументов
            self.request_parser(*self.url_generate(parsing_task)) if parsing_task != '' else print('Введен пустой запрос.')
        print('Список запросов пуст.') if len(self.task_array) <= 0 else print('Сбор данных завершен.')
        # закрываем браузер в селениум
        driver.quit()
        return self.df

# Работа с парсером
- Для запуска парсинга сайта [goldenpages.uz](https://www.goldenpages.uz/search/) требуется составить массив запросов поиска сайта, переменная task_array = ['кафе в самарканде', '', 'кафе в намангане', 'кафе в бухаре']. Ограничение сайта GoldenPages - запросы должны содержать минимум три слова, иначе вывод резултата поиска сайта будет пустым.
- Для старта парсинга и сохранения результипующего фрейма спарсенных данных требуется вызвать команду parsed_data = GoldenPagesParser(task_array).task_frame_parser() которая вернет Pandas Data Frame с результатом парсинга в переменную parsed_data. Код будет содержать следующие колонки собранных данных: id тип 'int' - номер собранной ячейки каждого запроса, company_name, company_second_name, company_third_name, address, landmarks, phone_array, site_url, parsing_url - сылка на страницу с которой были собраны данные, parsing_task - запрос при выполнении которого были собраны данные.
- Для сохранения результирующего фрейма в файл CSV требуется раскоментить строку кода parsed_data.to_csv(f"Selenium_Soup_GoldenPages_{datetime.now().strftime('%d%m%Y')}.csv") которая сохранить данные в файл с наименованием "Selenium_Soup_GoldenPages_01072024.csv" а папку с запускаемом кодом. Дата будет подставляться автоматически - текущая дата сохранения результата парсинга.

In [4]:
# массив запросов для парсинга, ограничение сайта GoldenPages - запросы должны содержать минимум три слова иначе вывод резултата поиска будет пустым
task_array = ['кафе в самарканде', '', 'кафе в намангане', 'кафе в бухаре']

# создаем экземпляр класса парсинга, вызываем парсинг тасков, и сохраняем фрейм данных одной строкой
parsed_data = GoldenPagesParser(task_array).task_frame_parser()

# сохраняем фрейм в файл текущей датой
#parsed_data.to_csv(f"Selenium_Soup_GoldenPages_{datetime.now().strftime('%d%m%Y')}.csv")

Page N:1, Task: кафе в самарканде
Page N:2, Task: кафе в самарканде
Page N:3, Task: кафе в самарканде
Введен пустой запрос.
Page N:1, Task: кафе в намангане
Page N:1, Task: кафе в бухаре
Сбор данных завершен.


In [5]:
display(parsed_data.info())
display(parsed_data.shape)
display(parsed_data.isnull().sum())
display(parsed_data[:50])

<class 'pandas.core.frame.DataFrame'>
Index: 62 entries, 0 to 61
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   62 non-null     int64 
 1   company_name         62 non-null     object
 2   company_second_name  8 non-null      object
 3   company_third_name   30 non-null     object
 4   address              62 non-null     object
 5   landmarks            45 non-null     object
 6   phone_array          62 non-null     object
 7   site_url             5 non-null      object
 8   parsing_url          62 non-null     object
 9   parsing_task         62 non-null     object
dtypes: int64(1), object(9)
memory usage: 5.3+ KB


None

(62, 10)

id                      0
company_name            0
company_second_name    54
company_third_name     32
address                 0
landmarks              17
phone_array             0
site_url               57
parsing_url             0
parsing_task            0
dtype: int64

,id,company_name,company_second_name,company_third_name,address,landmarks,phone_array,site_url,parsing_url,parsing_task
0,1,"""BBQ. BARBEQUE BURGER"" ФИЛИАЛ",None,BBQ. BARBEQUE BURGER - САМАРКАНД ATLAS,"140100, Узбекистан, Самаркандская область, Гор...","""ATLAS MEBEL"" ТОРГОВЫЙ КОМПЛЕКС ФИЛИАЛ, ТРЦ ""A...","[998915269666, 998955070006]",None,https://www.goldenpages.uz/search/?region=&s=к...,кафе в самарканде
1,2,"""BBQ. BARBEQUE BURGER"" ФИЛИАЛ",None,None,"140100, Узбекистан, Самаркандская область, Гор...","""FAMILY PARK"" ТОРГОВО-РАЗВЛЕКАТЕЛЬНЫЙ ЦЕНТР, Ш...","[998915269666, 998955070006]",None,https://www.goldenpages.uz/search/?region=&s=к...,кафе в самарканде
2,3,"""EVOS"" КАФЕ ФИЛИАЛ",None,"EVOS САМАРКАНД, БУЛЬВАР","140100, Узбекистан, Самаркандская область, Гор...","""AMAR REIKARTZ"" ГОСТИНИЦА, ""BOULEVARD PALACE"" ...","[998781291616, 998712035555]",None,https://www.goldenpages.uz/search/?region=&s=к...,кафе в самарканде
3,4,"""EVOS"" КАФЕ ФИЛИАЛ",None,"EVOS САМАРКАНД, МАРХАБО","140100, Узбекистан, Самаркандская область, Гор...","РЕСТОРАН ""МАРХАБО"", САМАРКАНДСКИЙ РУССКИЙ ТЕАТ...","[998781291616, 998712035555]",None,https://www.goldenpages.uz/search/?region=&s=к...,кафе в самарканде
4,5,"""EVOS"" КАФЕ ФИЛИАЛ",None,"EVOS САМАРКАНД, ДАГБИТСКИЙ","140100, Узбекистан, Самаркандская область, Гор...",САМАРКАНДСКОЕ ОБЛАСТНОЙ СОВЕТ ФЕДЕРАЦИИ ПРОФСО...,"[998781291616, 998712035555]",None,https://www.goldenpages.uz/search/?region=&s=к...,кафе в самарканде
5,6,"""EVOS"" КАФЕ ФИЛИАЛ",None,"EVOS САМАРКАНД, ДОШКОЛЬНЫЙ","140100, Узбекистан, Самаркандская область, Гор...","РЕСТОРАН ""RISOTTO"", ""JAHON PALACE"" ОТЕЛЬ","[998781291616, 998712035555]",None,https://www.goldenpages.uz/search/?region=&s=к...,кафе в самарканде
6,7,"""EVOS"" КАФЕ ФИЛИАЛ",None,"EVOS САМАРКАНД, МИР САИД БАРАКА","140100, Узбекистан, Самаркандская область, Гор...","""M. BARAKA"" ТОРГОВЫЙ ЦЕНТР, ""KORZINKA.UZ"" СУПЕ...","[998781291616, 998712035555]",None,https://www.goldenpages.uz/search/?region=&s=к...,кафе в самарканде
7,8,"""EVOS"" КАФЕ ФИЛИАЛ",None,"EVOS САМАРКАНД, ФРУНЗЕ","140100, Узбекистан, Самаркандская область, Гор...","ЗАГС №1 г.САМАРКАНДА, АРХИВ ЗАГСа САМАРКАНДСКО...","[998781291616, 998712035555]",None,https://www.goldenpages.uz/search/?region=&s=к...,кафе в самарканде
8,9,"""FAROVON CUISINE"" FAST FOOD",None,None,"140100, Узбекистан, Самаркандская область, Гор...","ЗАГС №4 г.САМАРКАНДА, ""NURAFSHON"" ТОРГОВО-РАЗВ...",[998782101000],farovon.uz,https://www.goldenpages.uz/search/?region=&s=к...,кафе в самарканде
9,10,"""GIOTTO"" КАФЕ","""VANILLA FOOD"" СП ООО","""GIOTTO"" - САМАРКАНД","140100, Узбекистан, Самаркандская область, Гор...","""FAMILY PARK"" ТОРГОВО-РАЗВЛЕКАТЕЛЬНЫЙ ЦЕНТР, Ш...",[998996709999],giotto.uz,https://www.goldenpages.uz/search/?region=&s=к...,кафе в самарканде
